In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
# %% [code]
# 데이터 불러오기
df = pd.read_csv("diabetes.csv")
df.head()
# %% [code]
# Regression: BMI 예측
# 특성: BMI를 제외한 모든 컬럼 (만약 Outcome과 같이 불필요한 컬럼이 있다면 추가로 제거)
features_reg = df.drop(columns=["BMI"])
target_reg = df["BMI"].values.reshape(-1, 1)

# 학습/테스트 데이터 분리
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    features_reg.values.astype(np.float32), target_reg.astype(np.float32),
    test_size=0.2, random_state=42
)

# 스케일링
scaler_reg = StandardScaler()
X_train_reg = scaler_reg.fit_transform(X_train_reg)
X_test_reg = scaler_reg.transform(X_test_reg)

# numpy -> PyTorch tensor 변환
X_train_reg_tensor = torch.tensor(X_train_reg)
X_test_reg_tensor = torch.tensor(X_test_reg)
y_train_reg_tensor = torch.tensor(y_train_reg)
y_test_reg_tensor = torch.tensor(y_test_reg)
# %% [code]
# Regression용 신경망 모델 정의
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)  # BMI 예측 (연속형 값)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim_reg = X_train_reg_tensor.shape[1]
model_reg = RegressionModel(input_dim_reg)

# 손실함수 및 옵티마이저: MSELoss 사용
criterion_reg = nn.MSELoss()
optimizer_reg = optim.Adam(model_reg.parameters(), lr=0.001)

# 학습
epochs_reg = 100
for epoch in range(epochs_reg):
    model_reg.train()
    optimizer_reg.zero_grad()
    outputs_reg = model_reg(X_train_reg_tensor)
    loss_reg = criterion_reg(outputs_reg, y_train_reg_tensor)
    loss_reg.backward()
    optimizer_reg.step()
    if (epoch+1) % 10 == 0:
        print(f"[Regression] Epoch {epoch+1}/{epochs_reg}, Loss: {loss_reg.item():.4f}")

# 평가
model_reg.eval()
with torch.no_grad():
    pred_reg = model_reg(X_test_reg_tensor)
    mse_reg = mean_squared_error(y_test_reg_tensor.numpy(), pred_reg.numpy())
    print(f"\nRegression Test MSE: {mse_reg:.4f}")


[Regression] Epoch 10/100, Loss: 1080.3663
[Regression] Epoch 20/100, Loss: 1070.1248
[Regression] Epoch 30/100, Loss: 1058.6816
[Regression] Epoch 40/100, Loss: 1044.1671
[Regression] Epoch 50/100, Loss: 1024.9843
[Regression] Epoch 60/100, Loss: 1000.1916
[Regression] Epoch 70/100, Loss: 968.8816
[Regression] Epoch 80/100, Loss: 930.4232
[Regression] Epoch 90/100, Loss: 884.1468
[Regression] Epoch 100/100, Loss: 829.7061

Regression Test MSE: 825.1093
